<a href="https://colab.research.google.com/github/flari-gold/IU/blob/main/IU_RedditDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#package installs with pip
!pip install praw

In [ ]:
#import modules
from google.colab import userdata
import praw
import collections

In [ ]:
#parameter
CLIENT_ID=userdata.get('CLIENT_ID')
CLIENT_SECRET=userdata.get('CLIENT_SECRET')
#PASSWORD=userdata.get('CLIENT_ID')
#USERNAME=userdata.get('CLIENT_ID')
SUBREDDIT='Sylosis'
ANZAHL_POSTS=30

In [ ]:
#initialisierung reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent="my user agent",
    check_for_async=False #PRAW wirft eine Warnung welche die ASYNC API empfiehlt, kann ignoriert werden
)

In [ ]:
#declaration
all_textdata = []
user_upvotes = collections.Counter()
user_contributionscounter = collections.Counter()


In [ ]:
#Anfragen an die Reddit API mit über request liefern die Daten im JSON Format. PRAW vereinfacht dies, in dem man die direkt die Attribute ansprechen kann
for submission in reddit.subreddit(SUBREDDIT).hot(limit=ANZAHL_POSTS):

    user_contributionscounter[submission.author] += 1
    user_upvotes[submission.author] += submission.score

    post_textdata = [submission.title]
    post_textdata.append(submission.selftext)


    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        post_textdata.append(comment.body)
        user_contributionscounter[comment.author] += 1
        user_upvotes[comment.author] += comment.score
        #print(comment.body)

    #print(post_textdata)
    all_textdata.append(post_textdata)
    #print("----------------------------------------------")


print(user_contributionscounter)
print(user_upvotes)
#print(all_textdata)

In [ ]:
#clean text
for i in all_textdata:
    #print(i)
    for j in i:
        print(j)


In [ ]:
subreddit = 'finanzen'
for submission in reddit.subreddit(subreddit).hot(limit=5):
    print(submission.title)
    print(submission.id)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        print(comment.body)
        print("----------------------------------------------")


#for submission in reddit.subreddit("test").hot(limit=10):
#    print(submission.title)
#    print(submission.id)

#submission = reddit.submission("1gmifua")
#submission.comments.replace_more(limit=None)
#for comment in submission.comments.list():
#    print(comment.body)
#    print("----------------------------------------------")

submission = reddit.submission("1gmifua")
print(submission.score)
comment = reddit.comment("lxw66nl")
print(comment.score)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 10, 100)

plt.plot(x, np.sin(x))
plt.plot(x, np.cos(x))

plt.show()

In [ ]:
!pip install HanTa
from HanTa import HanoverTagger as ht

tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')



In [ ]:
print(tagger_de.analyze('fährt'))

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

review_1 = "Loved the sound, no battery issues"
review_2 = "Sound quality is good; battery life not good"
vectorizer = TfidfVectorizer(use_idf = True,
	max_features = 5,
	smooth_idf=True)
model = vectorizer.fit_transform([review_1, review_2])

lda_model=LatentDirichletAllocation(n_components=2,learning_method=
'online',random_state=42,max_iter=1)
lda_top=lda_model.fit_transform(model)

print("Review 1: ")
for i,topic in enumerate(lda_top[0]):
	print("Topic ",i,": ",topic*100,"%")

for i,topic in enumerate(lda_top[1]):
	print("Topic ",i,": ",topic*100,"%")

In [ ]:
import requests



# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(userdata.get('CLIENT_ID'), userdata.get('CLIENT_SECRET'))

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': userdata.get('USERNAME'),
        'password': userdata.get('PASSWORD')}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [ ]:
subreddit = 'de'
limit = 50
timeframe = 'all' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top

url = f"https://oauth.reddit.com/r/{subreddit}/{listing}?limit={limit}&t={timeframe}"
res = requests.get(url, headers=headers)

import json
#print(json.dumps(res.json(), indent=2))



def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Import the wordcloud library
from wordcloud import WordCloud# Join the different processed titles together.
#long_string = ','.join(list(papers['paper_text_processed'].values))# Create a WordCloud object
long_string = ' '.join(find_values('title', json.dumps(res.json())))
#print(long_string)
templist = [word for word in long_string.split() if word.lower() not in stopwords.words('german')]
long_string = ' '.join(templist)

#

wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')# Generate a word cloud
wordcloud.generate(long_string)# Visualize the word cloud
wordcloud.to_image()



In [ ]:
subreddit = 'finanzen'
limit = 5
timeframe = 'week' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top
id = '1gmifua'

#http://www.reddit.com/r/" + sub + "/comments/" + id + ".json?",
url = f"https://oauth.reddit.com/r/{subreddit}/comments/{id}.json?"
res = requests.get(url, headers=headers)

import json
print(json.dumps(res.json(), indent=2))